In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 

from scipy import stats

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import OLSInfluence

from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro

In [2]:
tumanova = pd.read_csv(r"data/tum_preprocessing.csv")
puha = pd.read_csv(r"data/puha_preprocessing.csv")

In [3]:
tumanova.columns

Index(['Год выпуска процессора', 'Тактовая частота процессора(ГГц)',
       'Максимальная тактовая частота процессора(ГГц)',
       'Частота шины процессора(GT/s)', 'Объема кэша L2 процессора(Мб)',
       'Объема кэша L3 процессора(Мб)', 'Объем оперативной памяти (Гб)',
       'Частота оперативной памяти (МГц)', 'Количество ядер',
       'Количество потоков', 'Тип оперативной памяти',
       'Вид графического ускорителя', 'HDD', 'SSD',
       'Результа ты в бенчмарке PCMark10'],
      dtype='object')

In [4]:
puha.columns

Index(['Количество ядер', 'Количество логических процессоров (потоков)',
       'Тактовая частота процессора (ГГц)',
       'Максимальная тактовая частота (ГГц)', 'Объем кэша L2 процессора (Кб)',
       'Объем кэша L3 процессора (Кб)', 'Частота оперативной памяти (МГц)',
       'HDD (Гб)', 'SSD(Гб)', 'Объем видеопамяти (Гб) [дискретной]',
       'Объем видеопамяти (Гб) [встроенной]', 'Учебный', 'Развлекательный',
       'Эталон', 'Наличие графического ускорителя: дискретный',
       'Тип видеопамяти: DDR4', 'Тип видеопамяти: GDDR5',
       'Тип видеопамяти: GDDR6', 'Тип видеопамяти: LPDDR4X',
       'Тип видеопамяти: SMA', 'Оперативная память > 8', 'benchmark'],
      dtype='object')

## Модель

In [5]:
def get_std_coefs(X, y):
    res = [0]
    X_z = sm.add_constant(X.select_dtypes(include=[np.number]).dropna().apply(stats.zscore))
    Y_z = pd.DataFrame(y).apply(stats.zscore)
    model = sm.OLS(Y_z, X_z).fit()
    res.extend(model.params.tolist())
    return pd.Series(res)

In [6]:
def get_pivot(df, model, param, label, num):
    pivot_test = pd.DataFrame()
    predictors = model.params.index.tolist()
    n = len(predictors)
    i = np.ones(n).astype(int) * num
    pivot_test['id'] = pd.Series(i)
    pivot_test['predictors'] = predictors
    pivot_test['b'] = model.params.tolist()
    pivot_test['std_err'] = model.bse.tolist()
    influence = model.get_influence()
    pivot_test['std coef'] = get_std_coefs(df[param], df[label])
    pivot_test['t'] = model.tvalues.tolist()
    pivot_test['lower interval'] = model.conf_int(alpha=0.05)[:][0].tolist()
    pivot_test['upper interval'] = model.conf_int(alpha=0.05)[:][1].tolist()
    return pivot_test.set_index('id')

In [7]:
label = 'Результа ты в бенчмарке PCMark10'

In [8]:
data = tumanova

In [9]:
X_data = data.drop(label, axis=1)
Y_data = data.drop(X_data.columns, axis=1)

Y_stand = pd.DataFrame(data[label]).apply(stats.zscore)["Результа ты в бенчмарке PCMark10"]
X_stand = pd.DataFrame(data[{"Количество ядер","Объем оперативной памяти (Гб)","Тактовая частота процессора(ГГц)","Объема кэша L2 процессора(Мб)"}]).apply(stats.zscore)

Y = data[label]

In [10]:
predictors = data.drop(label, axis=1).to_dict()
sorted_predictors = sorted(predictors, key=lambda x: abs(data[x].corr(data[label])), reverse=True)

### Простая модель

In [11]:
pred_list = []
Y = data[label]
pivot_df = pd.DataFrame()
i = 1
for predictor in sorted_predictors:
    pred_list.append(predictor)
    X = sm.add_constant(data[pred_list])
    model = sm.OLS(Y, X).fit()
    F_crit = model.tvalues ** 2
    if F_crit[predictor] < 3.84:
        pred_list.pop()
    else:
        remove_list = list(F_crit[F_crit <= 2.71].index)
        for el in remove_list:
            if el in pred_list:
                pred_list.remove(el)
    if len(pred_list) != 0:
        model_info = get_pivot(data, model, pred_list, label, i)
        pivot_df = pd.concat([pivot_df, model_info], axis=0)
        i += 1

In [12]:
simple_model = model
pivot_simple = pivot_df

In [13]:
simple_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     Результа ты в бенчмарке PCMark10   R-squared:                       0.925
Model:                                          OLS   Adj. R-squared:                  0.902
Method:                               Least Squares   F-statistic:                     40.94
Date:                              Sat, 01 Oct 2022   Prob (F-statistic):           3.35e-10
Time:                                      22:43:53   Log-Likelihood:                -186.22
No. Observations:                                27   AIC:                             386.4
Df Residuals:                                    20   BIC:                             395.5
Df Model:                                         6                                         
Covariance Type:                          nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                           235.2792    206.498      1.139      0.268    -195.468     666.027
Количество ядер                 336.5130     67.919      4.955      0.000     194.837     478.189
Объем оперативной памяти (Гб)   521.3994    143.569      3.632      0.002     221.920     820.879
Частота шины процессора(GT/s)   152.2703     49.010      3.107      0.006      50.037     254.504
SSD                             287.3118    150.210      1.913      0.070     -26.020     600.644
Вид графического ускорителя     482.9900    191.308      2.525      0.020      83.928     882.052
HDD                            -333.8288    231.355     -1.443      0.165    -816.427     148.769
==============================================================================
Omnibus:                        0.143   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.931   Jarque-Bera (JB):                0.344
Skew:                           0.106   Prob(JB):                        0.842
Kurtosis:                       2.489   Cond. No.                         36.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### С фильтрацией

In [14]:
current_features = []
ignore_features = [label]
for i in range(10):
    function_dict = {'predictor': [], 'r-squared':[]}
    for predictor in sorted_predictors:
        if predictor not in (current_features+ignore_features):
            selected_X = data[current_features + [predictor]]
            model = sm.OLS(Y, sm.add_constant(selected_X)).fit()
            F_crit = model.tvalues ** 2
            #print(predictor, F_crit[predictor])
            if F_crit[predictor]>=3.84:
                #We add the predictor to our model
                #Now, we check if any other predictor's F is <2.71 and if so we drop em out
                remove_list = list(F_crit[F_crit <= 2.71].index)
                for el in remove_list:
                    if el in selected_X.columns:
                        selected_X.drop(el)
                model = sm.OLS(Y,sm.add_constant(selected_X)).fit()
                if len(model.params)>=3:
                    #Check the VIF
                    vif = pd.DataFrame()
                    vif["predictors"] = selected_X.columns
                    vif ["VIF values"] = [variance_inflation_factor(selected_X.values, i)
                    for i in range(len(selected_X.columns))]
                    if max(vif["VIF values"])<=10:
                        y_preds = model.predict(sm.add_constant(selected_X))
                        #Add the column name to our dictionary
                        function_dict['predictor'].append(predictor)
                        #Calculate the r-squared value between the target and predicted target
                        r2 = np.corrcoef(Y, y_preds)[0, 1]**2
                        #Add the r-squared value to our dictionary
                        function_dict['r-squared'].append(r2)
                else:
                    y_preds = model.predict(sm.add_constant(selected_X))
                    #Add the column name to our dictionary
                    function_dict['predictor'].append(predictor)
                    #Calculate the r-squared value between the target and predicted target
                    r2 = np.corrcoef(Y, y_preds)[0, 1]**2
                    #Add the r-squared value to our dictionary
                    function_dict['r-squared'].append(r2)
    if len(function_dict['predictor'])>0:
        #Once it's iterated through every column, turn our dict into a sorted DataFrame
        function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'],ascending = False)
        current_features.append(function_df["predictor"][0])
model = sm.OLS(Y, sm.add_constant(data[current_features])).fit()

In [15]:
filter_model = model
filter_pivot = pivot_df

In [16]:
filter_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     Результа ты в бенчмарке PCMark10   R-squared:                       0.902
Model:                                          OLS   Adj. R-squared:                  0.884
Method:                               Least Squares   F-statistic:                     50.36
Date:                              Sat, 01 Oct 2022   Prob (F-statistic):           9.20e-11
Time:                                      22:43:54   Log-Likelihood:                -189.84
No. Observations:                                27   AIC:                             389.7
Df Residuals:                                    22   BIC:                             396.2
Df Model:                                         4                                         
Covariance Type:                          nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                             -112.6890    321.320     -0.351      0.729    -779.066     553.688
Количество ядер                    516.9703     56.250      9.191      0.000     400.316     633.625
Объем оперативной памяти (Гб)      368.8835    153.940      2.396      0.025      49.631     688.136
Тактовая частота процессора(ГГц)   392.5724    126.833      3.095      0.005     129.536     655.609
SSD                                273.9732    133.261      2.056      0.052      -2.394     550.340
==============================================================================
Omnibus:                        0.536   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.765   Jarque-Bera (JB):                0.636
Skew:                           0.162   Prob(JB):                        0.728
Kurtosis:                       2.322   Cond. No.                         24.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### ВКР

In [17]:
tumanova_model = sm.OLS(Y, sm.add_constant(data[{"Объема кэша L2 процессора(Мб)","Количество ядер","Объем оперативной памяти (Гб)","Тактовая частота процессора(ГГц)"}])).fit()

In [18]:
tumanova_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     Результа ты в бенчмарке PCMark10   R-squared:                       0.902
Model:                                          OLS   Adj. R-squared:                  0.884
Method:                               Least Squares   F-statistic:                     50.64
Date:                              Sat, 01 Oct 2022   Prob (F-statistic):           8.71e-11
Time:                                      22:43:54   Log-Likelihood:                -189.77
No. Observations:                                27   AIC:                             389.5
Df Residuals:                                    22   BIC:                             396.0
Df Model:                                         4                                         
Covariance Type:                          nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                              -35.3972    311.384     -0.114      0.911    -681.169     610.374
Объема кэша L2 процессора(Мб)      388.9156    186.314      2.087      0.049       2.525     775.306
Объем оперативной памяти (Гб)      427.6965    148.162      2.887      0.009     120.427     734.966
Тактовая частота процессора(ГГц)   371.6294    124.340      2.989      0.007     113.763     629.495
Количество ядер                    445.9520     69.192      6.445      0.000     302.457     589.447
==============================================================================
Omnibus:                        1.553   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.460   Jarque-Bera (JB):                0.999
Skew:                           0.081   Prob(JB):                        0.607
Kurtosis:                       2.072   Cond. No.                         24.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Тесты

In [19]:
simple_model.params

const                            235.279193
Количество ядер                  336.513035
Объем оперативной памяти (Гб)    521.399370
Частота шины процессора(GT/s)    152.270275
SSD                              287.311766
Вид графического ускорителя      482.990048
HDD                             -333.828775
dtype: float64

### С фильтрацией

In [20]:
filter_model.params

const                              -112.688961
Количество ядер                     516.970266
Объем оперативной памяти (Гб)       368.883501
Тактовая частота процессора(ГГц)    392.572412
SSD                                 273.973161
dtype: float64

In [21]:
puha.loc[puha['SSD(Гб)'] > 0, 'SSD(Гб)'] = 1

In [22]:
puha_labels = ['Учебный', 'Развлекательный', 'Эталон']

In [23]:
res = puha[puha_labels]
filter_predict = filter_model.predict(sm.add_constant(puha[['Количество ядер', 'Оперативная память > 8', 'Тактовая частота процессора (ГГц)', 'SSD(Гб)']]))
for col in res.columns:
    res['Разность с ' + col] = res[col] - filter_predict


In [24]:
res

,Учебный,Развлекательный,Эталон,Разность с Учебный,Разность с Развлекательный,Разность с Эталон
0,5076,5082,5120,266.293463,272.293463,310.293463
1,2523,2299,2743,464.115960,240.115960,684.115960
2,4424,4371,4565,-1027.074657,-1080.074657,-886.074657
3,5052,5138,5204,517.094152,603.094152,669.094152
4,2176,2506,2561,-853.878420,-523.878420,-468.878420
5,2366,2331,2694,189.344236,154.344236,517.344236
6,5083,5101,5034,-760.647069,-742.647069,-809.647069
7,3255,3237,3542,1219.804871,1201.804871,1506.804871
8,3820,3757,3806,593.835373,530.835373,579.835373
9,3670,3719,3861,247.549167,296.549167,438.549167


In [25]:
model.params

const                              -112.688961
Количество ядер                     516.970266
Объем оперативной памяти (Гб)       368.883501
Тактовая частота процессора(ГГц)    392.572412
SSD                                 273.973161
dtype: float64

In [26]:
features = model.params.index.drop('const')

In [27]:
re_dict = {}
puha_filter = puha[['Количество ядер', 'Оперативная память > 8', 'Тактовая частота процессора (ГГц)', 'SSD(Гб)']]
p_cols = puha_filter.columns
for i in range(len(features)):
    re_dict[p_cols[i]] = features[i]

In [28]:
puha_filter = puha_filter.rename(columns=re_dict)

In [29]:
filter_dataset = pd.concat([tumanova[filter_model.params.index.drop('const')], puha_filter], axis=0)

In [30]:
filter_dataset

,Количество ядер,Объем оперативной памяти (Гб),Тактовая частота процессора(ГГц),SSD
0,6,1,2.2,0
1,2,1,1.9,0
2,2,0,2.0,0
3,2,1,2.2,1
4,2,1,1.8,1
5,4,1,2.4,1
6,2,0,2.6,0
7,2,1,2.6,0
8,4,1,1.6,1
9,4,0,2.3,0


In [31]:
from sklearn.metrics import r2_score

In [32]:
res = puha[puha_labels]
filter_predict = filter_model.predict(sm.add_constant(puha_filter))
res
for col in res.columns:
    res['r2 ' + col] = r2_score(res[col], filter_predict)

In [33]:
res

,Учебный,Развлекательный,Эталон,r2 Учебный,r2 Развлекательный,r2 Эталон
0,5076,5082,5120,0.667458,0.594804,0.5738
1,2523,2299,2743,0.667458,0.594804,0.5738
2,4424,4371,4565,0.667458,0.594804,0.5738
3,5052,5138,5204,0.667458,0.594804,0.5738
4,2176,2506,2561,0.667458,0.594804,0.5738
5,2366,2331,2694,0.667458,0.594804,0.5738
6,5083,5101,5034,0.667458,0.594804,0.5738
7,3255,3237,3542,0.667458,0.594804,0.5738
8,3820,3757,3806,0.667458,0.594804,0.5738
9,3670,3719,3861,0.667458,0.594804,0.5738


### Из ВКР

In [34]:
tumanova_model.params

const                               -35.397215
Объема кэша L2 процессора(Мб)       388.915553
Объем оперативной памяти (Гб)       427.696496
Тактовая частота процессора(ГГц)    371.629377
Количество ядер                     445.951950
dtype: float64

In [35]:
p_cols = ['Объем кэша L2 процессора (Кб)', 'Оперативная память > 8', 'Тактовая частота процессора (ГГц)', 'Количество ядер']
t_cols = tumanova_model.params.index.drop('const').tolist()


In [36]:
re_t = {}
for i in range(len(t_cols)):
    re_t[p_cols[i]] = t_cols[i]

In [37]:
tum_puha = puha[p_cols].rename(columns=re_t)

In [38]:
tum_puha['Объема кэша L2 процессора(Мб)'] = tum_puha['Объема кэша L2 процессора(Мб)'] / 1024

In [39]:
tum_puha

,Объема кэша L2 процессора(Мб),Объем оперативной памяти (Гб),Тактовая частота процессора(ГГц),Количество ядер
0,3.00,1,3.0,6
1,0.50,0,2.2,2
2,4.00,1,2.0,8
3,3.00,1,2.3,6
4,1.50,1,1.1,4
5,0.50,0,2.5,2
6,3.00,1,3.0,8
7,1.00,1,1.2,2
8,1.00,1,1.6,4
9,2.00,1,2.1,4


In [40]:
res2 = puha[puha_labels]

tumanova_predict = tumanova_model.predict(sm.add_constant(tum_puha))
res2['predict'] = tumanova_predict
for col in res2.columns:
    res2['r2 ' + col] = r2_score(res2[col], tumanova_predict)

In [41]:
res2

,Учебный,Развлекательный,Эталон,predict,r2 Учебный,r2 Развлекательный,r2 Эталон,r2 predict
0,5076,5082,5120,5349.645772,0.484302,0.409107,0.371253,1.0
1,2523,2299,2743,1868.549092,0.484302,0.409107,0.371253,1.0
2,4424,4371,4565,6258.835848,0.484302,0.409107,0.371253,1.0
3,5052,5138,5204,5089.505208,0.484302,0.409107,0.371253,1.0
4,2176,2506,2561,3168.272726,0.484302,0.409107,0.371253,1.0
5,2366,2331,2694,1980.037905,0.484302,0.409107,0.371253,1.0
6,5083,5101,5034,6241.549673,0.484302,0.409107,0.371253,1.0
7,3255,3237,3542,2119.073986,0.484302,0.409107,0.371253,1.0
8,3820,3757,3806,3159.629638,0.484302,0.409107,0.371253,1.0
9,3670,3719,3861,3734.359879,0.484302,0.409107,0.371253,1.0


In [42]:
tum_puha_dataset = pd.concat([tumanova[t_cols], tum_puha], axis=0)

In [43]:
tum_puha_dataset

,Объема кэша L2 процессора(Мб),Объем оперативной памяти (Гб),Тактовая частота процессора(ГГц),Количество ядер
0,1.00,1,2.2,6
1,0.50,1,1.9,2
2,0.50,0,2.0,2
3,0.50,1,2.2,2
4,0.50,1,1.8,2
5,1.00,1,2.4,4
6,0.50,0,2.6,2
7,0.50,1,2.6,2
8,1.00,1,1.6,4
9,1.00,0,2.3,4


### Общий тест

In [44]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [45]:
metrics = {'r2': r2_score,
           'MAE': mean_absolute_error,
           'MSE': mean_squared_error,
           'MAPE': mean_absolute_percentage_error}

In [46]:
print('Модель с фильтрацией\n')
for label in puha_labels:
    print(label)
    for metric in metrics.keys():
        print(f"{metric} = {metrics[metric](puha[label], filter_predict)}")
    print("\n")

Модель с фильтрацией

Учебный
r2 = 0.6674581203440026
MAE = 497.3441097032672
MSE = 354367.66307719494
MAPE = 0.17106591128464946


Развлекательный
r2 = 0.5948037225840157
MAE = 584.363826138934
MSE = 454283.67397420044
MAPE = 0.19692697600916118


Эталон
r2 = 0.5737997818060412
MAE = 562.2448245859714
MSE = 427241.52925862884
MAPE = 0.1701757488523923




In [47]:
print('Модель из ВКР\n')
for label in puha_labels:
    print(label)
    for metric in metrics.keys():
        print(f"{metric} = {metrics[metric](puha[label], tumanova_predict)}")
    print("\n")

Модель из ВКР

Учебный
r2 = 0.48430243520801963
MAE = 555.4557515599007
MSE = 549544.4395724807
MAPE = 0.19599017099092583


Развлекательный
r2 = 0.40910710410318607
MAE = 658.8328488421348
MSE = 662476.4605072606
MAPE = 0.22798783736673583


Эталон
r2 = 0.3712530756867717
MAE = 651.3178351970388
MSE = 630283.1063732448
MAPE = 0.2054444289359132




In [49]:
puha.to_csv("test_data.csv", index=False)